In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib



tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a=1
fluxes = []

for s,path_direction in enumerate([
                        "PNAS Test 2D Structures T(t) Samples N 480 xi 0.45 Localization/z_incidence",
                        "PNAS Test 2D Structures T(t) Samples N 480 xi 0.45 Diffusion/z_incidence",
                        
                        #"PNAS Test 2D Structures T(t) Samples N 480 xi 0.2 Localization 100 Runtime/z_incidence",
                        #"PNAS Test 2D Structures T(t) Samples N 480 xi 0.2 Diffusion/z_incidence"
                       ]):

      folder_path = f"data/{path_direction}"
      
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            print(filename)
            if i in [2,9]:
                  continue
            if not Path(filename).suffix==".txt":
                  continue
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                #print(structure_1.__str__())
                sim_data = structure_1.sim_data
                flux_t = sim_data["time_monitorT"].flux
                times = flux_t['t']
                max_index = np.argmax(flux_t.values)
                monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
                flux_item = {
                      "flux_t":flux_t,
                      "times":times,
                      "monitor_lambdas":monitor_lambdas,
                      "max_index":max_index,
                      "type":"Localization" if s == 0 else "Diffusive"
                }

                fluxes += [flux_item]
                
                plt.plot(times[:]*1e12,np.log(flux_t[:]/np.max(flux_t)),"o",label=f"$\\nu$: {1/monitor_lambdas[0]:.3g} - {1/monitor_lambdas[-1]:.3g}")
      

                

      plt.ylabel("Log(T(t))")
      plt.xlabel("Time[ps]")
      plt.title("Average transmission T(t)")
      plt.legend()
      plt.grid()
      plt.show()


In [ ]:
fluxes_loc =([np.array(item["flux_t"]) for item in fluxes if item["type"]=="Localization"])
fluxes_diff = ([np.array(item["flux_t"]) for item in fluxes if item["type"]=="Diffusive"])
times_diff = fluxes[-2]["times"]
times_loc = fluxes[0]["times"]
monitor_lambdas_diff=fluxes[-1]["monitor_lambdas"]
monitor_lambdas_loc=fluxes[0]["monitor_lambdas"]

In [ ]:
loc = np.abs((np.mean(fluxes_loc,axis=0)))
max_index = np.argmax(loc)
diff = np.abs(np.mean(fluxes_diff,axis=0))
max_index_diff = np.argmax(diff)

In [ ]:
def moving_average(x, w=3):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
bad_loc_data_log = np.where(np.isnan(np.log(loc)))
cleaned_loc_data_log = np.delete(loc,bad_loc_data_log)
cleaned_loc_data_time = np.delete(times_loc,bad_loc_data_log)

In [ ]:
bad_diff_data_log = np.where(np.isnan(np.log(diff)))
cleaned_diff_data_log = np.delete(diff,bad_diff_data_log)
cleaned_diff_data_time = np.delete(times_diff,bad_diff_data_log)

In [ ]:
loc_moving_avg = moving_average((cleaned_loc_data_log),25)
diff_moving_avg = moving_average((cleaned_diff_data_log),50)

In [ ]:
font = {
        'weight' : 'bold',
        'size'   : 27}

matplotlib.rc('font', **font)

In [ ]:
plt.figure(figsize=(15, 10))
times_avg=np.array(cleaned_loc_data_time[max_index+24:]*1e12)
loc_avg=np.log(loc_moving_avg[max_index:]/np.max(loc_moving_avg[max_index:]))
plt.plot(times_loc[max_index:]*1e12,np.log((loc[max_index:])/np.max((loc[max_index:]))),"o",label=f"$\\nu$: {1/monitor_lambdas_loc[0]:.3g} - {1/monitor_lambdas_loc[-1]:.3g}")
plt.plot(times_avg,loc_avg,label=f"Moving Average")
plt.ylabel("Log(T(t))")
plt.xlabel("Time[ps]")
plt.title("Average transmission T(t)")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
times_diff_avg=np.array(times_diff[max_index_diff+49:]*1e12)
diff_avg=np.log(diff_moving_avg[max_index_diff:]/np.max(diff_moving_avg[max_index_diff:]))
plt.plot(times_diff[max_index_diff:]*1e12,np.log(diff[max_index_diff:]/np.max(diff[max_index_diff:])),"o",label=f"$\\nu$: {1/monitor_lambdas_diff[0]:.3g} - {1/monitor_lambdas_diff[-1]:.3g}")
plt.plot(times_diff_avg,diff_avg,label=f"Moving Average")
plt.ylabel("Log(T(t))")
plt.xlabel("Time[ps]")
plt.title("Average transmission T(t)")
plt.legend()
plt.show()

In [ ]:
import jax
import jax.numpy as jnp
from jaxopt import ScipyMinimize
import jaxopt
import matplotlib.pyplot as plt
jax.config.update("jax_enable_x64", True)

In [ ]:
def diffussive(t,A):
    B = jnp.exp(-A*t)
    return jnp.log(B/jnp.max(B))

In [ ]:
#Anderson
Db = lambda l_var:3e2*l_var/3


def alpha_star(Db,xi,L):
    return (Db/xi**2)*jnp.exp(-L/xi)

def trans_t(t,alpha,td,p=0.5,s=0.85): 
    
    B = (alpha*td)**(s+1)*jnp.exp(-alpha*t+1)*(t*alpha)**-(p+1)
    #B =(s+1)*jnp.log(alpha*td) + (1-alpha*td) -(p+1)*jnp.log(alpha*t)
    return np.log(B/np.max(B))


def trans_t_1(t,alpha,p): 
    
    B = -p*jnp.log(t)-alpha*t

    return (B)

def transmission_anderson_func_jax(t,l,nu,L,td,s=0.85,p=0.5):
    Db_calculated = Db(l)
    k =  2*jnp.pi*(nu/(a))
    xi = 6*l*(k*l)**2/(1-(k*l)**4)
    print(xi/a)
    alpha= alpha_star(Db_calculated,xi,L)
    transmission = trans_t(t,alpha,td,p,s)
    return transmission
def transmission_anderson_func_jax_1(t,l,nu,L,td,s=0.85,p=0.5):
    Db_calculated = Db(l)
    k =  2*jnp.pi*(nu/(a))
    xi = 6*l*(k*l)**2/(1-(k*l)**4)
    alpha= alpha_star(Db_calculated,xi,L)
    transmission = trans_t_1(t,alpha,td,p,s)
    return transmission

In [ ]:
plt.figure(figsize=(15, 10))
model = lambda params,t: trans_t_1(t,params[0],params[1])
def loss_fn(params):
 y_pred = model(params, times_avg)
 return jnp.mean((y_pred - (loc_avg))**2)

initial_params = jnp.array([0.5,0.85], dtype=jnp.float32)
optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=20000)
# Optimize the parameters
result = optimizer.run(initial_params)
optimal_params = result.params

print("Anderson: ",optimal_params)

transmission_anderson = trans_t_1(times_avg,optimal_params[0],optimal_params[1])
plt.plot(times_avg,transmission_anderson,label="Anderson Fitted")


#####################Diff#########################
model = lambda params,t: diffussive(t,params[0])
def loss_fn(params):
 y_pred = model(params, times_avg[:])
 return jnp.mean((y_pred - loc_avg[:])**2)

initial_params = jnp.array([0.5], dtype=jnp.float32)
optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=15000)

# Optimize the parameters
result = optimizer.run(initial_params)
optimal_params = result.params

print("Diff: ",optimal_params)

transmission_diff = diffussive(times_avg,optimal_params[0])
plt.plot(times_avg,transmission_diff,label="Exponential Fit")

#################################################





plt.plot(times_avg,loc_avg,label=f"Moving Average")
plt.ylabel("Log(T(t))")
plt.xlabel("Time[ps]")
plt.legend()
plt.title("Average transmission T(t)")
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
indices = np.where(times_diff_avg<45)
model = lambda params,t: diffussive(t,params[0])
def loss_fn(params):
 y_pred = model(params, times_diff_avg[indices])
 return jnp.mean((y_pred - diff_avg[indices])**2)

initial_params = jnp.array([0.5], dtype=jnp.float32)
optimizer = jaxopt.BFGS(fun=loss_fn,maxiter=15000)

# Optimize the parameters
result = optimizer.run(initial_params)
optimal_params = result.params

print("Diff: ",optimal_params)

transmission_diff = diffussive(times_diff_avg,optimal_params[0])
plt.plot(times_diff_avg,transmission_diff,label="Diffusive Fitted")
plt.plot(times_diff_avg,diff_avg,label=f"Moving Average")
plt.ylabel("Log(T(t))")
plt.xlabel("Time[ps]")
plt.title("Average transmission T(t)")
plt.legend()
plt.grid()
plt.show()